In [1]:
import json
import numpy as np
import collections
import copy
from os import listdir
from os.path import isfile, join

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext
import pyspark
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '2'),('spark.executor.instances','7'), ('spark.driver.memory','32g'), ('spark.driver.maxResultSize','10g')])
sc = SparkContext(conf=conf)

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.types import Row
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [3]:
from operator import add

In [10]:
acsdb = sc.textFile("/srv/samba/group_workspace_1/deng.595/workspace/table_transformer/data/ACSDb/coheader_stats_10092007_uniquedByDomain.txt")

In [50]:
attr_stat = acsdb.filter(lambda x:x.startswith("single")).map(lambda x:x[7:].strip("_").split(" = "))
schema_stat = acsdb.filter(lambda x:x.startswith("combo")).map(lambda x:x[6:].strip("_").split(" = "))

In [47]:
from itertools import combinations

In [51]:
attr_stat = spark.createDataFrame(attr_stat.map(lambda x:(x[0],int(x[1]))), ["attribute","freq"])
schema_stat = spark.createDataFrame(schema_stat.map(lambda x:(list(combinations(x[0].split('_'),2)),int(x[1]))), ["attribute_pairs","freq"]).withColumn("schema_id", F.monotonically_increasing_id())

In [52]:
schema_stat = schema_stat.select("freq","schema_id",F.explode("attribute_pairs").alias("attribute_pair"))

In [54]:
attr_co_occ = schema_stat.groupBy("attribute_pair").agg(F.sum("freq").alias("co_freq"))

In [55]:
attr_co_occ.show()

+--------------------+-------+
|      attribute_pair|co_freq|
+--------------------+-------+
|           [1-4, 12]|    213|
|[back-spacing, of...|      2|
|    [attack, serve*]|    256|
|  [ends-:, status-:]|    337|
|[case-number, foe...|      1|
|[code, ext.-tube-...|      2|
|[code, max.-diffe...|      3|
|[invariant-cluste...|      1|
|          [cs2, cs3]|      3|
|            [7a, 9a]|     25|
|[tuesday, wednesday]|  35402|
|   [date, l.-island]|      4|
|[gb, lauro-memori...|      1|
|[business-phone, ...|      1|
|[avg-career-hbp, ...|      1|
|[%-rmsep, %-zero-...|      1|
|     [place, winner]|     41|
|        [mon:, thu:]|    304|
|         [rank, tds]|     60|
|[own-travel, sing...|      2|
+--------------------+-------+
only showing top 20 rows

